In [104]:
#!pip install dash_bootstrap_components

In [105]:
from dash import Dash, html, dash_table, dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import pandas as pd

Having installed the libraries you need, load up the provided csv file and have a quick look at the data by using print(df.head()).  This will give you the first 5 lines of data along with the column names.

Ensure you run the earlier cell prior to the cell below.

In [106]:
tutorial_data = pd.read_csv('dataanalysis.csv')


In our layout we want to show a map giving the unemployment rate by country followed by a line chart with selected countries showing.

First we need to format the data.  At the moment the data is in rows with a year for each row.  This is what we could call a "wide" data format.  We want to show the years for the countries, so that means transforming the data into a long format.  The pandas method of "melt" is used for us to reformat this data.  

Having reformatted our data, we create a choropleth map.  

In [107]:
formatted_df = pd.melt(tutorial_data,
                       ["Country Name", "Country Code"],
                       var_name="Year",
                       value_name="Unemployment Rate")
formatted_df = formatted_df.sort_values(by=["Country Name", "Year"])

# map
fig_map = px.choropleth(formatted_df, locations="Country Code",
                        color="Unemployment Rate",
                        hover_name="Country Name",
                        animation_frame="Year",
                        range_color=(0, 25),
                        color_continuous_scale='fall',
                        width=1000
                        )

fig_map.update_layout({
    'title_text': 'Global unemployment rate',
    'plot_bgcolor': 'rgb(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(176, 210, 161, 40)',
})


Our second pane will show a line graph where we have the countries and the unemployment rates.  We will not show ALL of the countries, but just a selection.  

In [108]:

# Create a line graph where the axes are year and unemployment rate.  Then only show three selected countries
fig_line = px.line(formatted_df, x="Year", y="Unemployment Rate", color='Country Name', color_discrete_sequence=px.colors.qualitative.Alphabet, width=1000,).update_traces(
    visible="legendonly", selector=lambda t: not t.name in ["Denmark", "Sweden", "Norway", "Finland"],)

# Here we format the way the graph looks by updating the axes with different shades.
fig_line.update_xaxes(tickangle=-45)
fig_line.update_xaxes(showline=True, linewidth=2, linecolor='black',
                      showgrid=True, gridwidth=1, gridcolor='lightGray')
fig_line.update_yaxes(showline=True, linewidth=2, linecolor='black',
                      showgrid=True, gridwidth=1, gridcolor='lightGray')


fig_line.update_layout({
    'title_text': 'Unemployment rate by country',
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})



In [109]:
# Identify the different panes of the dashboard that you want to visualise

# We want two panes of the dashboard, one with a map and the other with a line graph.

card_global_map = dbc.Card(
    [   
        dbc.CardHeader('Global Unemployment Rate',style={"background-color": "green", "color": "white"}),
        dbc.CardBody(
            [
                html.Div(id='global_unemp', className='card-value text-center', style={'font-size': '60px', 'font-weight': 'bold'}),
                dcc.Graph(figure=fig_map)
            ]
        )
    ], outline=True, color="success"
)

card_line_chart = dbc.Card(
    [
        dbc.CardHeader("Unemployment rate by country", style={"background-color": "blue", "color": "white"}),
        dbc.CardBody(
            [
                html.Div(id='line_map', className='card-value text-center', style={'font-size': '60px', 'font-weight': 'bold'}),
                dcc.Graph(figure=fig_line)
            ]
        )
    ], outline=True, color="primary"
)


Having created our graphs and created the cards where they will show, the next part is to create the dashboard component itself.  This is using Dash framework to create an HTML layout.  

In [110]:
# Initialize the app
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = html.Div([
    html.H1("Data Dashboard"),
    dbc.Row([
        # Note that the width adds up to 12 - the bootstrap grid system width
        dbc.Col(card_global_map, width=6),
        dbc.Col(card_line_chart, width=6)   
    ])
])

# Here we want the app to run in a browser tab external to the notebook but to also allow us to debug
if __name__ == '__main__':
    app.run(jupyter_mode="external", debug=True)
    





Dash app running on http://127.0.0.1:8050/
